In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import os
from utils import show_graph

In the mathematical theory of artificial neural networks, the universal approximation theorem states that a feed-forward network with a single hidden layer containing a finite number of neurons can approximate continuous functions on compact subsets of $\mathbb{R}^n$, under mild assumptions on the activation function.

Therefore, in theory there is no need to use more than one layer when we could just increase the number of units of the single hidden layer. In practice, for complex tasks, a MLP can achieve the same accuracy with a much faster training. However, multi-layer architectures introduce a few new issues during training which have to been handled and will be discussed in this chapter.

# Vanishing/exploding gradients problems
[Xavier and Yoshua (2010)](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf): "We can see that the variance of the gradient on the weights is the same for all layers, but the variance of the backpropagated gradient might still vanish or explode as we consider deeper networks. Note how this is reminiscent
of issues raised when studying recurrent neural networks (Bengio et al., 1994), which can be seen as very deep
networks when unfolded through time."

In the following subsection we will describe a few tricks to improve the stability of the gradients.

## Xavier and He intitialisation
Xavier and Yoshua (2010) and [Kaiming He et al (2015)](https://arxiv.org/pdf/1502.01852v1.pdf) give us weights initialisation techniques which provide a more stable variance of gradients and backpropagated gradient. If we denote with $n_{\text{input}}$ and $n_{\text{output}}$ the number of input and output connection respectively, then:
\begin{equation}
\text{Activation: Sigmoid}; W \sim U(-r, +r) \Longrightarrow r = \sqrt{\frac{6}{n_{\text{input}} + n_\text{output}}}
\end{equation}

\begin{equation}
\text{Activation: Hyperbolic tangent}; W \sim U(-r, +r) \Longrightarrow r = 4\sqrt{\frac{6}{n_{\text{input}} + n_\text{output}}}
\end{equation}

\begin{equation}
\text{Activation: ReLU}; W \sim U(-r, +r) \Longrightarrow r = \sqrt{2}\sqrt{\frac{6}{n_{\text{input}} + n_\text{output}}}
\end{equation}

\begin{equation}
\text{Activation: Sigmoid}; W \sim U(-r, +r) \Longrightarrow \sigma = \sqrt{\frac{2}{n_{\text{input}} + n_\text{output}}}
\end{equation}

\begin{equation}
\text{Activation: Hyperbolic tangent}; W \sim U(-r, +r) \Longrightarrow \sigma = 4\sqrt{\frac{2}{n_{\text{input}} + n_\text{output}}}
\end{equation}

\begin{equation}
\text{Activation: ReLU}; W \sim U(-r, +r) \Longrightarrow \sigma = \sqrt{2}\sqrt{\frac{2}{n_{\text{input}} + n_\text{output}}}
\end{equation}

In [2]:
he_init = tf.contrib.layers.variance_scaling_initializer(
    factor=2.0,
    mode='FAN_IN',
    uniform=False,
    seed=None,
    dtype=tf.float32,
)
#tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, kernel_initializer=he_init, name='hidden1')

## Nonsaturating activation functions
Sigmoid $\sigma(z)$ as activation function has the following issues:
- Gradient approaches to zero as |z| >> 0.
- Not centered around zero (see issues in Xavier and Yoshua (2010)).
- Non-linear everywhere, so not very fast to be computed.

Considerations with ReLU(z):
- Very wise to be computed because it's piecewise linear.
- Dying hidden units: If a hidden unit starts outputting zeros, there is no way to bring the unit back to live with a null gradient. You might find out that more then half of your hidden units die during training.

Extensions of ReLU(z):
- LeakyReLU(z) = $max(az, z)$: hidden units never die. $a$ is generally set to 0.01. Note that when $a$=1, your architecture becomes a plain logistic regression. Source: [Bing Xu et al. (2015)](https://arxiv.org/pdf/1505.00853.pdf)
- RandomizedLeakyReLU(z) = $max(az, z)$ where $a$ if randomly sampled during training and kept fixed during testing (similarly to dropout). It seems to perform well and acts as regulariser.
- ParametricLeakyRelu(z) = $max(az, z)$ where $a$ is treated like a parameter instead of a hyper-parameter, so $a$ is optimised by backpropagation. This increases the flexibility of your function, so there is a higher risk of overfitting.
- ExponentialLU(z) = $a(e^z - 1)$ if $z < 0$ else $z$. Source: [Clevert et al. (2016)](https://arxiv.org/pdf/1511.07289v5.pdf).
    - Considerations:
        - Slower than LReLU to be computed.
        - Average of the function tends to be closer to zero.
        - Smooth everywhere. Gradient does not jumps around zero.
        - Non-zero gradient for $z<0$, so hidden units don't die.


As a rule of thumb: ELU > LeakyReLU and variants > tanh > logistic. You can also try RReLU if you are underfitting or PReLU if overfitting.

## Batch normalisation
Batch normalisation is another tool in your arsenal to help with vanishing/exploding gradients.
[Sergey Ioffe (2015)](https://arxiv.org/pdf/1502.03167.pdf)


How it works:

This technique consists of adding an operation in the model just before the activation function of each layer, simply zero-centering and normalizing the inputs, then scaling the and shifting the result using two new parameters (\gamma, \beta) per layer. In other words, this operation lets the model learn the optimal scale and mean (\gamma, \beta) of each input for each layer.

\begin{equation}
\mu_B = \frac{1}{m_B} \sum_{i=1}^{m_B} \mathbf{x} ^ {(i)}
\end{equation}

\begin{equation}
\sigma_B^2 = \frac{1}{m_B} \sum_{i=1}^{m_B} \ (\mathbf{x} ^ {(i)} - \mu_B) ^ 2
\end{equation}

\begin{equation}
\hat{\mathbf{x}} ^ {(i)} = \frac{\mathbf{x} ^ {(i)} - \mu_B} {\sqrt{\sigma_B^2 + \epsilon}}
\end{equation}

\begin{equation}
\hat{\mathbf{z}} ^ {(i)} = \gamma \hat{\mathbf{x}} ^ {(i)} + \beta
\end{equation}
- At test time, empirical mean and standard deviation are taken from the whole training set.

Advantages:
- Vanishing/exploding gradients are strongly reduced, at the point that we can use tanh and even logistic.
- Network is much less sensitive to weight initialisation. However, weight initialisation is done offline so IMO this is not really a big advantage. However, this is still a good property.
- Training can require one order of magnitude less training steps.
- Batch normalisation acts as a regulariser, similarly to dropout.

Disadvantages:
- There is more logic involved with your network, more error-prone during the implementation.
- Slower predictions.

Generally batch normalization is great and make a lot of intuitive sense. The fact that it acts as a regularizer is great for datasets with low signal-to-noise ratio. It's also cool that it does not introduce and hyper-parameter.

In [3]:
# Load MNIST and split in training, validation and test set.
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Sizes of the dataset.
train_size, pixels_rows, pixels_cols = X_train.shape
test_size, _, _ = X_test.shape
nr_features = pixels_rows * pixels_cols
nr_labels = len(np.unique(y_train))

# split X_train in smaller X_train and X_valid.
split_threshold = int(train_size * 0.7)  
X_test = X_test.reshape(test_size, pixels_rows * pixels_cols) / 255.
X_train = X_train.reshape(train_size, pixels_rows * pixels_cols) / 255.
X_train, X_valid = X_train[:split_threshold], X_train[split_threshold:]
y_train, y_valid = y_train[:split_threshold], y_train[split_threshold:]

### Construction phase

Note that we don't specify any activation function for the fully connected layers because we want to apply the activation function after each batch normalization layer. However, many researchers argue that it is just as good, or even better, to place the batch normalization layers after (rather than before) the activation.

In my view, I tend to prefer applying batch normalization AFTER the activation so the gradient will be computed over normalized and unbiased quantities (a non-linear activation will shift the mean away from zero).

In [4]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=(None, nr_features), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')

training = tf.placeholder_with_default(input=False, shape=(), name='is_test')
with tf.name_scope('nn-architecture'):
    hidden_1 = tf.layers.dense(inputs=X, units=300, activation=None, name='hidden_layer_1')
    batch_norm_1 = tf.layers.batch_normalization(inputs=hidden_1, axis=-1, momentum=0.9, epsilon=0.001, name='batch_norm_1')
    hidden_2 = tf.layers.dense(inputs=tf.nn.elu(batch_norm_1), units=100, activation=None, name='hidden_layer_2')
    batch_norm_2 = tf.layers.batch_normalization(inputs=hidden_2, axis=-1, momentum=0.9, epsilon=0.001, name='batch_norm_2')
    output = tf.layers.dense(inputs=tf.nn.elu(batch_norm_2), units=nr_labels, activation=None, name='output_layer')
    logits = tf.layers.batch_normalization(inputs=output, axis=-1, momentum=0.9, epsilon=0.001, name='logits')

In [5]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=logits,
        name='cross_entropy',
    )
    cross_entropy_loss = tf.reduce_mean(cross_entropy)

In [6]:
with tf.name_scope('gradient_descent'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01, name='gradient_descent')

    # You could equivalently use training_op = optimizer.minimize(cross_entropy).
    gradients = optimizer.compute_gradients(cross_entropy)
    training_op = optimizer.apply_gradients(grads_and_vars=gradients, name='apply_gradient')

In [7]:
with tf.name_scope('validation'):
    is_correct_label_int = tf.nn.in_top_k(
        predictions=logits,
        targets=y,
        k=1,
        name='is_correct_label_vec',
    )
    is_correct_label_float = tf.cast(is_correct_label_int, tf.float32)
    accuracy = tf.reduce_mean(is_correct_label_float, name='accuracy')

In [8]:
show_graph(tf.get_default_graph())

### Execution phase
When using batch normalization, there are two differences during the execution phase.
- Training placeholder has to be set to true-
- Get the list of operations to run mean and standard deviation of the training set.

In [9]:
# Parameters
n_epochs = 10
batch_size = 50
batch_iters = len(X_train) // batch_size

In [10]:
saver = tf.train.Saver()

# Construction phase. Save summary statistics for Tensorboard.
dir_log = 'tf_logs/run_{}/'.format(datetime.utcnow().strftime('%Y%m%d_%H%M%S'))
summary_accuracy_train = tf.summary.scalar('Accuracy_train', accuracy)
summary_accuracy_valid = tf.summary.scalar('Accuracy_valid', accuracy)
file_writer = tf.summary.FileWriter(dir_log, graph=tf.get_default_graph())


extra_update_ops = tf.get_collection(key=tf.GraphKeys.UPDATE_OPS, scope=None)


with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_iter in range(batch_iters):
            batch_idx_start = batch_iter * batch_size
            batch_idx_end = (1 + batch_iter) * batch_size
            X_batch = X_train[batch_idx_start:batch_idx_end]
            y_batch = y_train[batch_idx_start:batch_idx_end]
            session.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_train = session.run(accuracy, feed_dict={X: X_train, y: y_train})
        accuracy_valid = session.run(accuracy, feed_dict={X: X_valid, y: y_valid})
        print('Epoch: {}; Training accuracy: {}; Validation accuracy: {}.'.format(epoch, accuracy_train, accuracy_valid))
        
        # Save model
        path_model = os.path.join('tf_checkpoints', 'session.ckpt')
        saver.save(session, path_model)
        
        # Save summary stats.
        file_writer.add_summary(
            summary=session.run(summary_accuracy_train, feed_dict={X: X_train, y: y_train}),
            global_step=epoch,
        )
        file_writer.add_summary(
            summary=session.run(summary_accuracy_valid, feed_dict={X: X_valid, y: y_valid}),
            global_step=epoch,
        )
        # Flush. See: https://github.com/tensorflow/tensorflow/issues/2353#issuecomment-287516024
        file_writer.flush()



Epoch: 0; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 1; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 2; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 3; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 4; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 5; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 6; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 7; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 8; Training accuracy: 0.0; Validation accuracy: 0.0.
Epoch: 9; Training accuracy: 0.0; Validation accuracy: 0.0.


In [ ]:
# TODO: Why is accuracy always zero_ Find the bug.

## Gradient clipping

# Reusing pre-trained layers

# Faster optimizers

# Avoiding overfitting through regularization

# Practical guidelines